In [1]:
#Student: Calvin Chu
#Super Freind Instructor: Ken Smith 
'''
Concepts 
State transition table / HMM
Viterbi algorithm

'''


import math

class HMMviterbi:
    def __init__(self,
                 observationList, 
                 stateNameList, 
                 startingProbabilityList, 
                 transitionProbabilityList, 
                 emissionProbabilityList):
        self.observationList = observationList
        self.stateNameList = stateNameList
        self.startingProbabilityList = startingProbabilityList
        self.transitionProbabilityList = transitionProbabilityList
        self.emissionProbabilityList = emissionProbabilityList
        self.viterbiTable = [{}] # Table for storing viterbi values and the previous state associated with each state at each timestep
    
    # Creates space for another entry in the viterbi table
    def newViterbiTableEntry(self):
        self.viterbiTable.append({})
    
    # Prints the contents of the viterbi table
    def printViterbiTable(self):
        numberOfObservations = len(self.observationList)
        for currentTimeStep in range(0, numberOfObservations):
            print(f"Time Step {currentTimeStep}")
            print(f"Observation:   {self.observationList[currentTimeStep]}")
            for eachState in self.stateNameList:
                print("")
                print(f"State Name:    {eachState}")
                print(f"Viterbi Value: {self.getViterbiValue(currentTimeStep, eachState)}")
                print(f"Previous State:{self.getPreviousState(currentTimeStep, eachState)}")
            print("_____________________")
    
    # Returns the highest viterbi value in the viterbi table along with the corresponding state
    def getMaxFinalViterbiValueAndState(self):
        lastTimeStep = len(self.observationList)-1
        maxViterbiValue = -1
        maxVertibiState = None
        for eachState in self.stateNameList:
            thisViterbiValue = self.getViterbiValue(lastTimeStep, eachState)
            if thisViterbiValue > maxViterbiValue:
                maxViterbiValue = thisViterbiValue
                maxViterbiState = eachState
        return maxViterbiValue, maxViterbiState
    
    # Starting from the given timeStep and the given state,
    # traces the state sequence in the viterbi table
    # returns a list of viterbi values and a list of state names
    # lists are in chronological order
    def backTrackForViterbiAndStateSequenceList(self, timeStep, state):
        stateSequence = [state]
        viterbiSequence = [self.getViterbiValue(timeStep, state)]
        while(timeStep > 0):
            prevState = self.getPreviousState(timeStep, state)
            stateSequence.insert(0, prevState)
            
            state = prevState
            timeStep-=1
            
            prevViterbi = self.getViterbiValue(timeStep, state)
            viterbiSequence.insert(0, prevViterbi)
        return viterbiSequence, stateSequence
            
    # Returns probability of getting targetObservation at targetState
    def getEmissionProbability(self, targetObservation, targetState):
        return self.emissionProbabilityList[targetState][targetObservation]
    
    # Returns probability of transitioning from startState to endState
    def getTransitionProbability(self, startState, endState):
        return self.transitionProbabilityList[startState][endState]
    
    # Returns probability of starting at targetState
    def getStartingStateProbability(self, targetState):
        return self.startingProbabilityList[targetState]
    
    # Returns the probability of starting at targetState and producing the output of targetObservation
    def getFirstObservationProbability(self, targetObservation, targetState):            
        probabilityOfStartingAtThisState = self.startingProbabilityList[targetState]
        probabilityOfGettingThisEmissionAtThisState = self.getEmissionProbability(targetObservation, targetState)
        return probabilityOfStartingAtThisState * probabilityOfGettingThisEmissionAtThisState
    
    # Returns the viterbi value for the given timeStep and the targetState
    def getViterbiValue(self, timeStep, targetState):
        return self.viterbiTable[timeStep][targetState]["probability"]
    
    # Returns the previous state for the given timeStep and the targetState
    def getPreviousState(self, timeStep, targetState):
        return self.viterbiTable[timeStep][targetState]["previous"]
    
    # Performs the viterbi algorithm given the data stored upon class instantiation
    # prints the most likely state sequence given a sequence of observations
    # set printTable to True to also print contents of viterbi table
    def performViterbi(self, printTable = False):
        # Fill data in viterbi table for the first timeStep
        currentTimeStep = 0
        currentObservation = self.observationList[currentTimeStep]
        previousState = None
        for eachState in self.stateNameList:
            firstObservationProb = self.getFirstObservationProbability(currentObservation, eachState)
            self.viterbiTable[0][eachState] = {"probability": firstObservationProb, "previous": previousState}
        
        # Fill data in viterbi table for the remaining timeSteps
        numberOfObservations = len(self.observationList)
        for currentTimeStep in range(1, numberOfObservations):
            self.newViterbiTableEntry()
            
            # Goes through each state and finds the best path to that state
            for eachState in self.stateNameList:
                # Use first state in stateNameList as the default for the best previous state
                defaultPreviousState = self.stateNameList[0]
                previousTimeStep = currentTimeStep-1
                previousViterbiValue = self.getViterbiValue(previousTimeStep, defaultPreviousState)
                transitionProbToTargetState = self.getTransitionProbability(defaultPreviousState, eachState)
                
                bestTransitionProbSoFar = previousViterbiValue * transitionProbToTargetState
                previousStateForBestTransitionProb = defaultPreviousState
                
                # Finds a state that provides the best previous state
                numberOfStates = len(self.stateNameList)
                for previousStateIndex in range(1, numberOfStates):
                    previousState = self.stateNameList[previousStateIndex]
                    previousViterbiValue = self.getViterbiValue(previousTimeStep, previousState)
                    transitionProbToTargetState = self.getTransitionProbability(previousState, eachState)
                    
                    maybeBestTransitionProbSoFar = previousViterbiValue * transitionProbToTargetState
                    maybePreviousStateForBestTransitionProb = previousState
                    
                    if maybeBestTransitionProbSoFar > bestTransitionProbSoFar:
                        bestTransitionProbSoFar = maybeBestTransitionProbSoFar
                        previousStateForBestTransitionProb = maybePreviousStateForBestTransitionProb
                
                # Fills viterbi table with new data
                currentObservation = self.observationList[currentTimeStep]
                viterbiValue = bestTransitionProbSoFar * self.getEmissionProbability(currentObservation, eachState)
                self.viterbiTable[currentTimeStep][eachState] = {"probability": viterbiValue, "previous": previousStateForBestTransitionProb}            
        
        if printTable:
            self.printViterbiTable()
        
        maxViterbiValue, maxViterbiState = self.getMaxFinalViterbiValueAndState()
        print(f"Score: {math.log(maxViterbiValue)}") #Prints score value
        
        # Backtracks through the completed viterbi table and finds the best state sequence
        finalTimeStep = numberOfObservations-1
        viterbiList, stateList = self.backTrackForViterbiAndStateSequenceList( finalTimeStep, maxViterbiState)
        
        # Print results
        print("Given observations:")
        for eachItem in self.observationList:
            print(eachItem)
        print("")
        print("The most likely state sequence is:")
        for i in range(0, len(viterbiList)):
            print(f"state:{stateList[i]}  viterbi:{viterbiList[i]}")
        
    

In [2]:
observationData = ('NoWalk', 'NoWalk', 'NoWalk', 'Walk') # What you observed happen in sequence
stateNames = ('Beg', 'NoBeg') # Possible states
probabilityOfStartingAtEachState = {'Beg': 0.5, 'NoBeg': 0.5}
probabilityOfTranstitioningFromEachState = {
   'Beg' : {'Beg': 0.1, 'NoBeg': 0.9},
   'NoBeg' : {'Beg': 0.2, 'NoBeg': 0.8}
   }
probabilityOfEmittingSomethingAtEachState = {
   'Beg' : {'Walk': 0.7, 'NoWalk': 0.3},
   'NoBeg' : {'Walk': 0.1, 'NoWalk': 0.9}
   }

viterbi = HMMviterbi(observationData, 
                     stateNames,
                     probabilityOfStartingAtEachState,
                     probabilityOfTranstitioningFromEachState,
                     probabilityOfEmittingSomethingAtEachState)
viterbi.performViterbi(printTable = True)

Time Step 0
Observation:   NoWalk

State Name:    Beg
Viterbi Value: 0.15
Previous State:None

State Name:    NoBeg
Viterbi Value: 0.45
Previous State:None
_____________________
Time Step 1
Observation:   NoWalk

State Name:    Beg
Viterbi Value: 0.027000000000000003
Previous State:NoBeg

State Name:    NoBeg
Viterbi Value: 0.32400000000000007
Previous State:NoBeg
_____________________
Time Step 2
Observation:   NoWalk

State Name:    Beg
Viterbi Value: 0.019440000000000002
Previous State:NoBeg

State Name:    NoBeg
Viterbi Value: 0.23328000000000004
Previous State:NoBeg
_____________________
Time Step 3
Observation:   Walk

State Name:    Beg
Viterbi Value: 0.032659200000000006
Previous State:NoBeg

State Name:    NoBeg
Viterbi Value: 0.018662400000000006
Previous State:NoBeg
_____________________
Score: -3.4216286865346763
Given observations:
NoWalk
NoWalk
NoWalk
Walk

The most likely state sequence is:
state:NoBeg  viterbi:0.45
state:NoBeg  viterbi:0.32400000000000007
state:NoBeg  v